# Ejercicio_3: Modelo previsión de fallas

In [241]:
#Llamado de librerias

import pandas as pd

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, roc_auc_score

In [227]:
# Subir los dos CSV a la consola y visuliazar los datos
df_2 = pd.read_csv('full_devices.csv',encoding='ISO-8859-1')

In [229]:
# Revisar las caracteristicas del DF
df_2.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 124494 entries, 0 to 124493
Data columns (total 12 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   date        124494 non-null  object
 1   device      124494 non-null  object
 2   failure     124494 non-null  int64 
 3   attribute1  124494 non-null  int64 
 4   attribute2  124494 non-null  int64 
 5   attribute3  124494 non-null  int64 
 6   attribute4  124494 non-null  int64 
 7   attribute5  124494 non-null  int64 
 8   attribute6  124494 non-null  int64 
 9   attribute7  124494 non-null  int64 
 10  attribute8  124494 non-null  int64 
 11  attribute9  124494 non-null  int64 
dtypes: int64(10), object(2)
memory usage: 11.4+ MB


In [231]:
# Cambiar de formato a data para que quede del tipo date_time

df_2['date']= pd.to_datetime(df_2['date'])

In [233]:
df_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 124494 entries, 0 to 124493
Data columns (total 12 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   date        124494 non-null  datetime64[ns]
 1   device      124494 non-null  object        
 2   failure     124494 non-null  int64         
 3   attribute1  124494 non-null  int64         
 4   attribute2  124494 non-null  int64         
 5   attribute3  124494 non-null  int64         
 6   attribute4  124494 non-null  int64         
 7   attribute5  124494 non-null  int64         
 8   attribute6  124494 non-null  int64         
 9   attribute7  124494 non-null  int64         
 10  attribute8  124494 non-null  int64         
 11  attribute9  124494 non-null  int64         
dtypes: datetime64[ns](1), int64(10), object(1)
memory usage: 11.4+ MB


In [235]:
# Organizar de forma cronologica y por device

df_2.sort_values(by=['device','date'], inplace = True)

In [153]:
# Vista con cambios

df_2.head()

,date,device,failure,attribute1,attribute2,attribute3,attribute4,attribute5,attribute6,attribute7,attribute8,attribute9
0,2015-01-01,S1F01085,0,215630672,56,0,52,6,407438,0,0,7
1163,2015-01-02,S1F01085,0,1650864,56,0,52,6,407438,0,0,7
2326,2015-01-03,S1F01085,0,124017368,56,0,52,6,407438,0,0,7
3489,2015-01-04,S1F01085,0,128073224,56,0,52,6,407439,0,0,7
4651,2015-01-05,S1F01085,0,97393448,56,0,52,6,408114,0,0,7


In [157]:
# Crear campo para identificar el dìa anterior a la falla

df_2['failure_next_day'] = df_2.groupby('device')['failure'].shift(-1)

In [119]:
# Vista con cambios

df_2.head()

,date,device,failure,attribute1,attribute2,attribute3,attribute4,attribute5,attribute6,attribute7,attribute8,attribute9,failure_next_day
0,2015-01-01,S1F01085,0,215630672,56,0,52,6,407438,0,0,7,0.0
1163,2015-01-02,S1F01085,0,1650864,56,0,52,6,407438,0,0,7,0.0
2326,2015-01-03,S1F01085,0,124017368,56,0,52,6,407438,0,0,7,0.0
3489,2015-01-04,S1F01085,0,128073224,56,0,52,6,407439,0,0,7,0.0
4651,2015-01-05,S1F01085,0,97393448,56,0,52,6,408114,0,0,7,0.0


In [167]:
# Comprobación de DF filtrado

#df_2 = df_2[df_2['failure_next_day'] == 1]

# Quitar valores nulos de la columna creada

df_2.dropna(subset=['failure_next_day'], inplace=True)


In [171]:
# Dividir datos por para entrnar el modelo y otro paraprobarlo, para efectos del ejercio deje la relación 80% - 20%

train_size = int(len(df_2) * 0.8)
train = df_2[:train_size]
test = df_2[train_size:]

# Separar características (X) y variable objetivo (y)

X_train = train.drop(columns=['failure', 'failure_next_day', 'date', 'device'])
y_train = train['failure_next_day']

X_test = test.drop(columns=['failure', 'failure_next_day', 'date', 'device'])
y_test = test['failure_next_day']

In [239]:
# Entrenar el modelo por medio del metodo Random Forest

model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Hacer predicción

y_pred = model.predict(X_test)
y_pred_prob = model.predict_proba(X_test)[:, 1] 



In [237]:
# Validar modelo con AUC

print(classification_report(y_test, y_pred))
print("AUC-ROC: ", roc_auc_score(y_test, y_pred_prob))

              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00     24645
         1.0       0.00      0.00      0.00        20

    accuracy                           1.00     24665
   macro avg       0.50      0.50      0.50     24665
weighted avg       1.00      1.00      1.00     24665

AUC-ROC:  0.7788770541692027


In [187]:

# Subir archivo prueba

df_3 = pd.read_excel('full_device_test.xlsx')

In [188]:
df_3.head()

,date,device,failure,attribute1,attribute2,attribute3,attribute4,attribute5,attribute6,attribute7,attribute8,attribute9
0,2016-01-01,S1F01085,NaN,215630672,56,0,52,6,407438,0,0,7
1,2016-01-01,S1F0166B,NaN,61370680,0,3,0,6,403174,0,0,0
2,2016-01-01,S1F01E6Y,NaN,173295968,0,0,0,12,237394,0,0,0
3,2016-01-01,S1F01JE0,NaN,79694024,0,0,0,6,410186,0,0,0
4,2016-01-01,S1F01R2B,NaN,135970480,0,0,0,15,313173,0,0,3


In [193]:

# cCmbio de df de prueba

X_new = df_3[['attribute1', 'attribute2', 'attribute3', 'attribute4', 'attribute5', 'attribute6', 'attribute7', 'attribute8', 'attribute9']]

# Predicción de la falla
y_new_pred = model.predict(X_new)

# Probabilidad de la falla
y_new_pred_prob = model.predict_proba(X_new)[:, 1]

In [203]:

# Crear dt con probabilidad y predicción

df_new_predictions = pd.DataFrame({
    'device': df_3['device'],
    'date': df_3['date'],
    'probabilidad_falla': y_new_pred_prob,
    'prediccion_falla': y_new_pred
})

# Mostrar las primeras filas de las predicciones
print(df_new_predictions.head(100))




      device       date  probabilidad_falla  prediccion_falla
0   S1F01085 2016-01-01                 0.0               0.0
1   S1F0166B 2016-01-01                 0.0               0.0
2   S1F01E6Y 2016-01-01                 0.0               0.0
3   S1F01JE0 2016-01-01                 0.0               0.0
4   S1F01R2B 2016-01-01                 0.0               0.0
..       ...        ...                 ...               ...
95  S1F0GKFX 2016-01-01                 0.0               0.0
96  S1F0GKL6 2016-01-01                 0.0               0.0
97  S1F0GPFZ 2016-01-01                 0.0               0.0
98  S1F0GPXY 2016-01-01                 0.0               0.0
99  S1F0GQCL 2016-01-01                 0.0               0.0

[100 rows x 4 columns]


In [243]:

# Aplicar filtro para comprobar

df_new_predictions_filter = df_new_predictions[df_new_predictions['prediccion_falla'] == 1]

df_new_predictions_filter.head(100)



,device,date,probabilidad_falla,prediccion_falla
3724,S1F0RRB1,2016-01-04,0.73,1.0
4885,S1F0RRB1,2016-01-05,0.64,1.0
5856,S1F0CTDN,2016-01-06,0.65,1.0
6171,S1F10E6M,2016-01-06,0.66,1.0
6213,S1F11MB0,2016-01-06,0.70,1.0
...,...,...,...,...
105437,S1F0T2LA,2016-07-22,0.76,1.0
107252,W1F14XGD,2016-07-27,0.60,1.0
107840,S1F135TN,2016-07-29,0.60,1.0
108293,W1F1DQN8,2016-07-30,0.66,1.0


In [215]:
df_new_predictions_filter.info()

<class 'pandas.core.frame.DataFrame'>
Index: 124389 entries, 0 to 124493
Data columns (total 4 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   device              124389 non-null  object        
 1   date                124389 non-null  datetime64[ns]
 2   probabilidad_falla  124389 non-null  float64       
 3   prediccion_falla    124389 non-null  float64       
dtypes: datetime64[ns](1), float64(2), object(1)
memory usage: 4.7+ MB
